In [6]:
import pandas as pd

In [80]:
import pandas as pd

pd.options.display.max_columns = 50

# Load the CSV files into DataFrames
users_df = pd.read_csv('../prep/PREP_Users.csv')
courses_df = pd.read_csv('../prep/PREP_Courses.csv')
training_performances_df = pd.read_csv('../prep/PREP_TrainingPerformances.csv')
feedbacks_df = pd.read_csv('../prep/PREP_Feedbacks.csv')


In [8]:
# Join users and training performances on employeeID
user_training_df = pd.merge(users_df, training_performances_df, left_on='id', right_on='employeeID')

# Join the result with courses on courseID
user_training_courses_df = pd.merge(user_training_df, courses_df, left_on='courseID', right_on='id')

# Join users and feedbacks on employeeID
user_feedback_df = pd.merge(users_df, feedbacks_df, left_on='employeeID', right_on='employeeID')


In [38]:
df = user_training_df.copy()
df['row'] = df.sort_values(by='attempt', ascending=False) \
              .groupby(['employeeID_x', 'courseID']) \
              .cumcount() + 1

result_df = df[df['row'] == 1]

user_training_final_df = result_df.drop(columns=['row'])

user_training_courses_final_df = pd.merge(user_training_final_df, courses_df, left_on='courseID', right_on='id')

In [90]:
#fact table for datascience
datascience_table = pd.merge(user_training_courses_final_df, user_feedback_df[['employeeID', 'aggregatedScore']], left_on='employeeID_x', right_on='employeeID', how='outer').fillna(0)


datascience_table['employee_id'] = datascience_table.apply(
    lambda row: row['employeeID'] if row['employeeID_x'] == 0 else row['employeeID_x'], axis=1
)

aggregated_data = datascience_table.groupby(['employee_id', 'designation_x']).agg(
    avg_training_score=('score', 'mean'),
    no_of_trainings=('course', 'count'),
    feedback_score=('aggregatedScore', 'mean')
).reset_index()

aggregated_data.rename(
    columns={
        'designation_x': 'designation'
    },
    inplace=True
)

final_df = aggregated_data[['employee_id', 'designation', 'no_of_trainings', 'avg_training_score', 'feedback_score']]

final_df.to_csv('DS_TABLE.csv', index=False)

print(final_df.head(10))

  employee_id          designation  no_of_trainings  avg_training_score  \
0      JMD327  Principle Architect                2                50.0   
1      JMD363    Software Engineer                4                35.0   
2      JMD596                    0                1                 0.0   
3      JMD617                    0                1                 0.0   

   feedback_score  
0            20.0  
1           100.0  
2            20.0  
3            20.0  


In [87]:
datascience_table.head(10)

,Unnamed: 0_x,id_x,name_x,email,password,employeeID_x,designation_x,reportingManager,userType,Unnamed: 0_y,id_y,employeeID_y,name_y,courseID,course,score,time,attempt,createdAt,updatedAt,Unnamed: 0,id,title,duration,designation_y,description,questions,startCourseDate,courseExpireDate,employeeID,aggregatedScore
0,3.0,4.0,Prasad,prasad@gmail.com,$2a$10$oZBJEzIKH7KvyNWVrq7dSu3Huji3kSWfQ/P/.9Q...,JMD327,Principle Architect,JMD364,employee,6.0,10.0,4.0,Prasad,104.0,Cloud,100.0,12.0,1.0,2024-10-07 04:46:59.723000+00:00,2024-10-07 04:46:59.723000+00:00,2.0,104.0,Cloud,1.0,Principle Architect,This is a course on Cloud for Principle Archit...,"[{'answers': ['Answer 1', 'Answer 2', 'Answer ...",2024-10-03 00:00:00+00:00,2025-10-03 00:00:00+00:00,JMD327,20.0
1,3.0,4.0,Prasad,prasad@gmail.com,$2a$10$oZBJEzIKH7KvyNWVrq7dSu3Huji3kSWfQ/P/.9Q...,JMD327,Principle Architect,JMD364,employee,7.0,11.0,4.0,Prasad,109.0,Nodejs,0.0,1.0,1.0,2024-10-07 04:47:10.200000+00:00,2024-10-07 04:47:10.200000+00:00,7.0,109.0,Nodejs,1.0,Principle Architect,This is a course on Nodejs for Principle Archi...,"[{'answers': ['Answer 1', 'Answer 2', 'Answer ...",2024-10-03 00:00:00+00:00,2025-10-03 00:00:00+00:00,JMD327,20.0
2,0.0,1.0,Aswath,bharaniaswath.rm@gmail.com,$2a$10$fjRdqTuhVUi1XWGpqRrdq.chBg49WGLR7J4rIC0...,JMD363,Software Engineer,JMD364,employee,2.0,6.0,1.0,Aswath,119.0,Java,60.0,29.0,2.0,2024-10-04 05:28:58.807000+00:00,2024-10-04 05:28:58.807000+00:00,17.0,119.0,Java,5.0,Software Engineer,This is a course on Java for Software Engineer.,[{'answers': ['A function within another funct...,2024-10-03 00:00:00+00:00,2025-10-03 00:00:00+00:00,JMD363,100.0
3,0.0,1.0,Aswath,bharaniaswath.rm@gmail.com,$2a$10$fjRdqTuhVUi1XWGpqRrdq.chBg49WGLR7J4rIC0...,JMD363,Software Engineer,JMD364,employee,3.0,7.0,1.0,Aswath,117.0,QA,40.0,10.0,1.0,2024-10-04 11:42:03.314000+00:00,2024-10-04 11:42:03.314000+00:00,15.0,117.0,QA,5.0,Software Engineer,This is a course on QA for Software Engineer.,[{'answers': ['A function within another funct...,2024-10-03 00:00:00+00:00,2025-10-03 00:00:00+00:00,JMD363,100.0
4,0.0,1.0,Aswath,bharaniaswath.rm@gmail.com,$2a$10$fjRdqTuhVUi1XWGpqRrdq.chBg49WGLR7J4rIC0...,JMD363,Software Engineer,JMD364,employee,4.0,8.0,1.0,Aswath,135.0,Data Science,20.0,16.0,2.0,2024-10-04 13:09:37.540000+00:00,2024-10-04 13:09:37.540000+00:00,33.0,135.0,Data Science,5.0,Software Engineer,This is a course on Data Science for Software ...,[{'answers': ['A function within another funct...,2024-10-03 00:00:00+00:00,2025-10-03 00:00:00+00:00,JMD363,100.0
5,0.0,1.0,Aswath,bharaniaswath.rm@gmail.com,$2a$10$fjRdqTuhVUi1XWGpqRrdq.chBg49WGLR7J4rIC0...,JMD363,Software Engineer,JMD364,employee,5.0,9.0,1.0,Aswath,127.0,Javascript,20.0,9.0,1.0,2024-10-04 13:09:57.968000+00:00,2024-10-04 13:09:57.968000+00:00,25.0,127.0,Javascript,5.0,Software Engineer,This is a course on Javascript for Software En...,[{'answers': ['A function within another funct...,2024-10-03 00:00:00+00:00,2025-10-03 00:00:00+00:00,JMD363,100.0
6,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0,0,JMD596,20.0
7,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0,0,JMD617,20.0


In [39]:
#Average Score per Designation:
avg_score_per_designation = user_training_courses_final_df.groupby('designation_x')['score'].mean()

avg_score_per_designation.to_csv("REPORT_avg_score_per_designation.csv")

In [27]:
#Course Completion Rate by Employee:
course_completion_count = user_training_courses_df.groupby('employeeID_x')['courseID'].nunique()

course_completion_count.to_csv('REPORT_course_completion_count.csv')

In [40]:
#Top Courses by Average Score:
top_courses_by_score = user_training_courses_final_df.groupby('title')['score'].mean().sort_values(ascending=False)

top_courses_by_score.to_csv('REPORT_top_courses_by_score.csv')

In [20]:
#Average Time Spent on Courses by Designation:
avg_time_spent = user_training_courses_df.groupby('designation_x')['time'].mean()

avg_time_spent.to_csv('REPORT_avg_time_spent.csv')

In [77]:
# #Top-Performing Employees (Based on Scores):
# top_performing_employees = user_training_courses_final_df.groupby('employeeID_x')['score'].mean().sort_values(ascending=False)

# top_performing_employees.to_csv('REPORT_top_performing_employees.csv')

top_performing_employees = (
    user_training_courses_final_df.groupby(['employeeID_x', 'name_x', 'designation_x'])
    .agg({'score': 'mean', 'time': 'mean'})
    .reset_index()
)

# Sort by the 'score' column in descending order
top_performing_employees = top_performing_employees.sort_values(by='score', ascending=False)
top_performing_employees.to_csv('REPORT_top_performing_employees.csv')

In [76]:
user_training_courses_final_df.columns

Index(['Unnamed: 0_x', 'id_x', 'name_x', 'email', 'password', 'employeeID_x',
       'designation_x', 'reportingManager', 'userType', 'Unnamed: 0_y', 'id_y',
       'employeeID_y', 'name_y', 'courseID', 'course', 'score', 'time',
       'attempt', 'createdAt', 'updatedAt', 'Unnamed: 0', 'id', 'title',
       'duration', 'designation_y', 'description', 'questions',
       'startCourseDate', 'courseExpireDate'],
      dtype='object')

In [43]:
#Training Attempts by Employee:
avg_attempts_per_employee = user_training_courses_df.groupby('employeeID_x')['attempt'].mean()

avg_attempts_per_employee.to_csv('REPORT_avg_attempts_per_employee.csv')

In [44]:
#Feedback Scores by Manager:
avg_feedback_by_manager = user_feedback_df.groupby('managerId')['aggregatedScore'].mean()

avg_feedback_by_manager.to_csv('REPORT_avg_feedback_by_manager.csv')

In [45]:
#Distribution of Feedback Scores:
feedback_score_distribution = user_feedback_df['aggregatedScore'].value_counts()

feedback_score_distribution.to_csv('REPORT_feedback_score_distribution.csv')

In [46]:
#Employee Count by Designation:
employee_count_by_designation = users_df['designation'].value_counts()

employee_count_by_designation.to_csv('REPORT_employee_count_by_designation.csv')

In [55]:
#Courses by Designation:
courses_per_designation = courses_df['designation'].value_counts()

courses_per_designation.to_csv('REPORT_courses_per_designation.csv')

In [57]:
#Feedback Scores by Designation:
avg_feedback_score_by_designation = user_feedback_df.groupby('designation_x')['aggregatedScore'].mean()

avg_feedback_score_by_designation.to_csv('REPORT_avg_feedback_score_by_designation.csv')

In [62]:
#Most Popular Courses (by Enrollment):
popular_courses = user_training_courses_final_df['title'].value_counts()

popular_courses.to_csv('REPORT_popular_courses.csv')

In [65]:
#Average Time Taken per Course:
avg_time_per_course = user_training_courses_df.groupby('title')['time'].mean()

avg_time_per_course.to_csv('REPORT_avg_time_per_course.csv')

In [66]:
#Most Feedback Given by Manager:
feedback_count_by_manager = user_feedback_df['managerId'].value_counts()

feedback_count_by_manager.to_csv('REPORT_feedback_count_by_manager.csv')

In [75]:
import numpy as np
user_training_courses_final_df1 = user_training_courses_final_df.groupby('employeeID_x')['score'].mean()
merged_scores_df = pd.merge(user_training_courses_final_df1, user_feedback_df[['employeeID', 'aggregatedScore']], left_on='employeeID_x', right_on='employeeID', how='outer').fillna(0)
merged_scores_df['Final_score'] = (merged_scores_df['score'] + merged_scores_df['aggregatedScore'])/2
merged_scores_df['Retained'] = np.where(merged_scores_df['Final_score'] > 50, 'Retained', 'Not Retained')

merged_scores_df.to_csv('REPORT_retained_table.csv')